# Visualize Laser Scan

SPDX-License-Identifier: MIT  
SPDX-FileCopyrightText: 2022 Daisuke Sato

https://github.com/Tiryoh/mini-pupper-jupyter-notebooks

## Set Network configuration

In [1]:
# NETWORK_INTERFACE="eth0"
NETWORK_INTERFACE="wlan0"

import netifaces as ni
ip_address = ni.ifaddresses(NETWORK_INTERFACE)[ni.AF_INET][0]['addr']
print(ip_address)

192.168.10.23


In [2]:
import os

# Set self ip address
os.environ['ROS_IP'] = ip_address

# Set ROS_MASTER_URI
# ROS_MASTER_IP=ip_address
ROS_MASTER_IP="192.168.10.14" # custom

os.environ['ROS_MASTER_URI'] = 'http://{}:11311'.format(ROS_MASTER_IP)

### Check environment  

It should outputs like this:

```
192.168.10.23
http://192.168.10.14:11311
```

If the result is `http://localhost:11311`, re-run the above procedure.

In [3]:
# Check Env
!echo $ROS_IP
!echo $ROS_MASTER_URI

192.168.10.23
http://192.168.10.14:11311


## Import libraries

In [4]:
from sidecar import Sidecar
# jupyros(jupyter-ros) 0.6.0 is not availabe on conda for now
try:
    from jupyros import ros3d
except ImportError:
    from jupyros.ros1 import ros3d

# Check jupyros version
import jupyros
print(jupyros.__version__)

0.5.0


## Create View

The output should look like this:

<img src="https://i.gyazo.com/f5092c17bc8753f6c278346e48901b96.png" width="400px" />

In [19]:
grid = ros3d.GridModel()
grid.color = '#CCC'
viewer = ros3d.Viewer()
viewer.layout.height= '1000px'
viewer.objects = [grid]

try:
    sc = Sidecar(title='Sidecar Output')
    with sc:
        display(viewer)
except:
    display(viewer)

## Subscribe Laser Scan

### Check topic

If it shows `ERROR: Unable to communicate with master!` or `ERROR: Unknown topic /scans`, re-run the network configuration step.

In [6]:
!rostopic info /scan

Type: sensor_msgs/LaserScan

Publishers: 
 * /LD06 (http://192.168.10.14:43479/)

Subscribers: 
 * /rosbridge_websocket (http://192.168.10.14:46601/)




### View scan topic

The output should look like this:

<img src="https://i.gyazo.com/db3430f1648fc9a30df1facc87339f75.png" width="400px" />

Reference
* https://jupyter-ros.readthedocs.io/en/latest/reference.html

In [20]:
ros_connection = ros3d.ROSConnection(url="ws://{}:9090".format(ROS_MASTER_IP))
tf_client = ros3d.TFClient(ros=ros_connection, fixed_frame="base_link")
laser_view = ros3d.LaserScan(topic="/scan", ros=ros_connection, tf_client=tf_client)
urdf_view = ros3d.URDFModel(ros=ros_connection, tf_client=tf_client, url='http://{}:3000'.format(ROS_MASTER_IP))
laser_view.point_size = 0.02
laser_view.static_color = "red"

viewer.objects = [grid, laser_view, urdf_view]

In [ ]:
!rosnode info rosbridge_websocket